# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 

from sklearn.metrics import classification_report, accuracy_score

In [50]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [52]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import Birch
from sklearn.metrics.cluster import fowlkes_mallows_score

score = []
for i in range(len(X.columns)): # loop number of features
  K = Birch(n_clusters=len(Y[Y.columns[0]].unique()))
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = fowlkes_mallows_score(Y[Y.columns[0]].values,pred)
  score.append(s)
  


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (4) by Birch is less than (9). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (6) by Birch is less than (9). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (2) by Birch is less than (9). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/birch.py:627: ConvergenceWarning: Number of subclusters found (2) by Birch is less than (9). Decrease the threshold.
  % (len(centroids), self.n_clusters), ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/birch.py:627: Converg

In [53]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , fowlkes_mallows_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , fowlkes_mallows_score

(9, 0.8749375133406012)
(32, 0.7848758646734929)
(1, 0.7848197753371678)
(6, 0.7803805243934263)
(14, 0.7800959020266208)
(15, 0.7800908073922788)
(28, 0.7766142323088678)
(30, 0.7734103792483976)
(17, 0.7674346483100938)
(16, 0.7614765747421549)
(0, 0.7355938628152945)
(29, 0.7298514642264796)
(5, 0.7190812053976248)
(19, 0.7157412249506845)
(18, 0.7115391859843534)
(33, 0.7080169273592045)
(2, 0.6985574144733353)
(34, 0.6942741606030993)
(35, 0.6942741606030993)
(23, 0.693803148959462)
(22, 0.6937956940942739)
(11, 0.6937160393901104)
(27, 0.6937050681981516)
(26, 0.6935701763873146)
(12, 0.6925181742217382)
(7, 0.6925062828493994)
(3, 0.6925045806720647)
(13, 0.6924572181387113)
(20, 0.6923700596059192)
(31, 0.6923536279882361)
(8, 0.6923501884951507)
(4, 0.6923501702349553)
(21, 0.6922324748970036)
(24, 0.6843422304916076)
(25, 0.6843422304916076)
(10, 0.6617785109522026)
(39, 0.48990609404133845)
(42, 0.44227169417228707)
(37, 0.4399826434

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [56]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6500    0.1680    0.2670       619
      Backdoor     0.6410    0.0882    0.1550       567
           DoS     0.3127    0.2655    0.2872      4075
      Exploits     0.6326    0.8088    0.7100     11236
       Fuzzers     0.9006    0.8654    0.8826      6127
       Generic     0.9972    0.9857    0.9914     53886
Reconnaissance     0.9234    0.7474    0.8261      3385
     Shellcode     0.6714    0.6108    0.6397       388
         Worms     0.9231    0.3158    0.4706        38

      accuracy                         0.8904     80321
     macro avg     0.7391    0.5395    0.5811     80321
  weighted avg     0.8942    0.8904    0.8876     80321



In [57]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [58]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.8918250151734441 max accuracy
11 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [60]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [61]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.19005876108119563)
('service', 0.1446559331792762)
('smeansz', 0.13649813575933334)
('Sjit', 0.07393343861558589)
('proto', 0.06999945270609652)
('dbytes', 0.05891972618691914)
('dmeansz', 0.04773817995745024)
('dur', 0.04292528562689194)
('Sintpkt', 0.042802341392846756)
('Dpkts', 0.037699753029574366)
('state', 0.02907261762994326)
('sttl', 0.01740784914689214)
('sloss', 0.012823715167112296)
('Dload', 0.012316927180890967)
('cprtt', 0.009741704558055135)
('Spkts', 0.009314344118141209)
('synack', 0.009237535256937908)
('dloss', 0.008963038916256357)
('Djit', 0.008938638960054576)
('Dintpkt', 0.007797943662421846)
('ackdat', 0.007147709319970341)
('stcpb', 0.006456953267328418)
('dtcpb', 0.006259828962369732)
('ct_flw_http_mthd', 0.004794213389741931)
('trans_depth', 0.002150404026950014)
('dttl', 0.0009099315521991027)
('dwin', 0.0004943464284051936)
('ct_state_ttl', 0.0004691144600874709)
('swin', 0.0003895610529350528)
('ct_ftp_cmd', 4.900490584758304e-05)
('is_ftp_lo

In [62]:
clf.score(x_train, y_train) 

0.9250737279497016

In [63]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.8918250151734441

In [64]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.6667    0.1495    0.2442       495
      Backdoor     0.6364    0.0896    0.1570       469
           DoS     0.3583    0.1759    0.2360      3235
      Exploits     0.6163    0.8713    0.7220      8944
       Fuzzers     0.8630    0.8320    0.8472      4846
       Generic     0.9969    0.9870    0.9919     43219
Reconnaissance     0.9310    0.7452    0.8278      2696
     Shellcode     0.5587    0.3707    0.4457       321
         Worms     0.7692    0.3125    0.4444        32

      accuracy                         0.8918     64257
     macro avg     0.7107    0.5037    0.5462     64257
  weighted avg     0.8914    0.8918    0.8836     64257

